In [ ]:
import getpass
from arcgis.gis import *

password = getpass.getpass("Please enter password: ")
dev_gis = GIS('https://www.arcgis.com', 'johnnyDev', password)
print("Successfully logged in to {} as {}".format(dev_gis.properties.urlKey + '.' + dev_gis.properties.customBaseUrl,
                                                 dev_gis.users.me.username))

In [ ]:
feature_layer_srch_results = dev_gis.content.search(query='title: "Griffith*" AND type: "Feature Service"', 
                                         max_items=10)
feature_layer_srch_results

In [ ]:
feature_layer_coll_item = feature_layer_srch_results[0]
feature_layers = feature_layer_coll_item.layers
feature_layer = feature_layers[0]
feature_layer.properties.name

In [ ]:
for field in feature_layer.properties['fields']:
    print(field['name'])

In [ ]:
from arcgis import geometry
from arcgis import features

def create_feature(map1, g):
    try:
        pt = geometry.Point(g)
        feat = features.Feature(geometry=pt, attributes={'OBJECTID': 1,
                                                        'name': 'name',
                                                        'type': 'park',
                                                        'surface': 'dirt'})
        feature_layer.edit_features(adds=[feat])
        print(str(g))
        map1.draw(g)
    except:
        print("Couldn't create the feature. Try again, please...")

In [ ]:
map1 = dev_gis.map('Los Angeles', 10)
map1

In [ ]:
map1.on_click(create_feature)

In [ ]:
map1.clear_graphics()

In [ ]:
map1.add_layer(feature_layer)